In [18]:
import pandas as pd
import imp
import pprint
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.font_manager import FontProperties
%matplotlib qt
fontP = FontProperties()
fontP.set_size('xx-small')

In [64]:
imp.reload(eval)

settings = {
        'csvs': ["my"],
        'bet_on': ['under', 'over'],
        'kelly_range': [0.1, 0.2, 0.1],
        'start': 1,
        'verb': True,
        'kelly_up_lim': 10,
        'kelly_low_lim': -1,
        'pred_up_lim': 1,
        'pred_low_lim': 0,
}

e = eval.daily(settings)
ret = e.eval()


# Plot
fig = plt.figure()
plt.title('ROI')
plt.xlabel('Potential bets (over and under for each player in each game)')
plt.ylabel('ROI (if start = 1 else just return)')
plt.grid()
plots=[]

# Plot each day
plots.append(plt.plot(ret['my-0.1']['data']['days'], ret['my-0.1']['data']['days_return'], label = f"{csv}_days_{pre}")[0])

# Plot each bet
x = list(range(0, ret['my-0.1']['total_bets']))
plots.append(plt.plot(x, ret['my-0.1']['data']['bets_return'], label = f"{csv}_bets_{pre}")[0])

plt.legend(handles=plots, title='Lines', bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
plt.show()

Csv: "my.csv" and Pre: 0.1
Ending 2021-05-24 with 111.72% of starting capital
Ending 2021-05-24 with 125.36% of starting capital
Ending 2021-05-24 with 136.88% of starting capital
Ending 2021-05-24 with 128.72% of starting capital
Ending 2021-05-24 with 176.18% of starting capital
Ending 2021-05-24 with 246.36% of starting capital
Ending 2021-05-24 with 277.56% of starting capital
Ending 2021-05-24 with 325.59% of starting capital
Ending 2021-05-24 with 419.18% of starting capital
Ending 2021-05-24 with 413.95% of starting capital
Ending 2021-05-24 with 415.17% of starting capital
Ending 2021-05-24 with 401.0% of starting capital
Ending 2021-05-24 with 469.4% of starting capital
Ending 2021-05-24 with 271.58% of starting capital
Ending 2021-05-24 with 285.1% of starting capital
Ending 2021-05-24 with 348.68% of starting capital
Ending 2021-05-24 with 321.32% of starting capital
Ending 2021-05-24 with 247.53% of starting capital
Ending 2021-05-24 with 223.86% of starting capital
betted 

In [55]:
fig = plt.figure()
plt.title('ROI')
plt.xlabel('Potential bets (over and under for each player in each game)')
plt.ylabel('ROI (if start = 1 else just return)')
plt.grid()
plots=[]

# Vars to change
start = 1 # 1 to see accual roi
pre_pre = 0 # How much risk to take, reasonable values range from 0.25 to 0.01
csvs = ["my"]#, "my_o"]

for pre in np.arange(0.1,0.2,0.1):
    pre = pre.round(decimals=2)
    if pre_pre:
        pre=pre_pre
    for csv in csvs:
        df = pd.read_csv(f"./{csv}.csv", sep=";")
        df['kelly_under'] = ((df['odds_under'] - 1) * df['pred_under'] - df['pred_over']) / (df['odds_under'] - 1)
        df['kelly_under'] = ((df['odds_under'] - 1) * df['pred_under'] - (1-df['pred_under'])) / (df['odds_under'] - 1)
        df['kelly_over'] = ((df['odds_over'] - 1) * df['pred_over'] - df['pred_under']) / (df['odds_over'] - 1) 
        df['kelly_over'] = ((df['odds_over'] - 1) * df['pred_over'] - (1-df['pred_over'])) / (df['odds_over'] - 1) 

        days = {}

        for index, row in df.iterrows():
            if str(row['date']) not in days:
                days[str(row['date'])] = {
                        'bets': [{
                            "date": row['date'],
                            "answer": row['answer'],
                            "pred_under": row['pred_under'],
                            "pred_over": row['pred_over'],
                            "odds_over": row['odds_over'],
                            "odds_under": row['odds_under'],
                            "kelly_under": row['kelly_under'],
                            "kelly_over": row['kelly_over']
                        }]
                    }
            else:
                days[str(row['date'])]['bets'].append(
                        {
                            "date": row['date'],
                            "answer": row['answer'],
                            "pred_under": row['pred_under'],
                            "pred_over": row['pred_over'],
                            "odds_over": row['odds_over'],
                            "odds_under": row['odds_under'],
                            "kelly_under": row['kelly_under'],
                            "kelly_over": row['kelly_over']
                        }
                    )

        num_day = 0
        keys = list(days.keys())

        tot_bets=0
        tot_bets_betted=0
        tot_bets_won=0
        plot_days=[0]
        plot_rois_day=[1]
        plot_rois_bet=[]

        # Loop trought days
        for day, info in days.items():
            if num_day == 0:
                info['start'] = start
            else:
                info['start'] = days[keys[num_day-1]]['end']
            if info['start'] > 0:
                info['end'] = info['start']

                # Loop through games in this day
                for bet in info['bets']:
                    for o_u in ["under"]:
                        # Do i want to bet on this? (is kelly saying bet)
                        if bet[f"kelly_{o_u}"] > 0.1:
                            if bet[f"pred_{o_u}"] > 0.5:
                                # Yes, how much should i bet?
                                how_much_to_bet = bet[f"kelly_{o_u}"] * pre * info['start']

                                # Remove from "kassa"
                                info['end'] -= how_much_to_bet

                                # Did i win? if so add how much i won
                                if (bet['answer'] == (o_u == "over")):
                                    info['end'] += how_much_to_bet * bet[f"odds_{o_u}"]
                                    tot_bets_won+=1
                                tot_bets_betted+=1

                        tot_bets += 1
                        plot_rois_bet.append(info['end'])

                plot_rois_day.append(info['end'])
                plot_days.append(plot_days[-1] + len(info['bets']*2))

                num_day += 1


        print(f"Csv: \"{csv}.csv\" and Pre: {pre}")
        for days, info in days.items():
            print(f"Ending {day} with {round(info['end']*100,2)}% of starting capital")
        print(f"betted {tot_bets_betted}/{tot_bets} = {round((tot_bets_betted / tot_bets)*100, 3)}% games")
        print(f"with a winrate of {round((tot_bets_won / tot_bets_betted)*100, 3)}%")
        print("")

        #plots.append(plt.plot(plot_days, plot_rois_day, label = f"{csv}_days_{pre}")[0])
        x = list(range(0, tot_bets))
        plots.append(plt.plot(x, plot_rois_bet, label = f"{csv}_bets_{pre}")[0])
    if pre_pre:
        break

plt.legend(handles=plots, title='Lines', bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
plt.show()

Csv: "my.csv" and Pre: 0.1
Ending 2021-05-24 with 103.64% of starting capital
Ending 2021-05-24 with 104.56% of starting capital
Ending 2021-05-24 with 105.48% of starting capital
Ending 2021-05-24 with 105.02% of starting capital
Ending 2021-05-24 with 104.14% of starting capital
Ending 2021-05-24 with 114.4% of starting capital
Ending 2021-05-24 with 113.41% of starting capital
Ending 2021-05-24 with 121.78% of starting capital
Ending 2021-05-24 with 120.55% of starting capital
Ending 2021-05-24 with 119.29% of starting capital
Ending 2021-05-24 with 114.76% of starting capital
Ending 2021-05-24 with 114.76% of starting capital
Ending 2021-05-24 with 123.08% of starting capital
Ending 2021-05-24 with 117.18% of starting capital
Ending 2021-05-24 with 116.2% of starting capital
Ending 2021-05-24 with 115.67% of starting capital
Ending 2021-05-24 with 120.87% of starting capital
Ending 2021-05-24 with 114.98% of starting capital
Ending 2021-05-24 with 103.68% of starting capital
betted